# House Price CV & Baseline Model

In [ ]:
from typing import Dict, Set
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.feature_selection import mutual_info_regression
import seaborn as sns
import warnings
plt.style.use('ggplot')
pd.options.display.max_columns = None
pd.options.display.max_rows = 30
warnings.filterwarnings('ignore', message='use_inf_as_na option is deprecated')

In [ ]:
data = pd.read_csv('../input/train.csv', index_col='Id') 
data_test = pd.read_csv('../input/test.csv', index_col='Id')
data = data[sorted(data)]
X_raw = data.drop(columns=['SalePrice'])
y = data.SalePrice

In [ ]:
def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values()
    return mi_scores


def group_features(df: pd.DataFrame, ordinal_cutoff: int=10) -> Dict[str, Set]:
    columns = set(df)
    cat_columns = set(df.select_dtypes(include=object))
    ordinal_cols = []
    cont_cols = []
    for column in columns - cat_columns:
        if df[column].nunique() < ordinal_cutoff:
            ordinal_cols.append(column)
        else:
            cont_cols.append(column)
    return {
        "nominal": cat_columns, 
        "ordinal": set(ordinal_cols), 
        "continuous": set(cont_cols)
    }


def clean(df: pd.DataFrame, features: Dict[str, Set]) -> pd.DataFrame:
    num_columns = features['continuous']
    df_cont = df[list(num_columns)].copy()
    df_cont = df_cont.fillna(df_cont.mean())
    assert all(df_cont.isnull().sum() == 0)
    
    # TODO: make this configurable since ordinal encdoing doesn't work well with linear models
    encoder = OrdinalEncoder().set_output(transform="pandas")
    df_ord = encoder.fit_transform(df[list(features['ordinal'])])
    
    encoder = OneHotEncoder(sparse_output=False).set_output(transform="pandas")
    df_cat = encoder.fit_transform(df[list(features['nominal'])])
    
    df_clean = pd.concat([df_cont, df_ord, df_cat], axis=1)
    has_nans = [c for c in df_clean if df_clean[c].isnull().any()]
    return df_clean.drop(columns=has_nans)

In [ ]:
# feature_groups = dict(
#     continuous=[
#         '1stFlrSF',
#          '2ndFlrSF',
#          '3SsnPorch',
#          'BedroomAbvGr',
#          'BsmtFinSF1',
#          'BsmtFinSF2',
#          'BsmtFullBath',
#          'BsmtHalfBath',
#          'BsmtUnfSF',
#          'EnclosedPorch',
#          'Fireplaces',
#          'FullBath',
#          'GarageArea',
#          'GarageCars',
#          'GarageYrBlt',
#          'GrLivArea',
#          'HalfBath',
#          'KitchenAbvGr',
#          'LotArea',
#          'LotFrontage',
#          'LowQualFinSF',
#          'MasVnrArea',
#          'MiscVal',
#          'OpenPorchSF',
#          'PoolArea',
#          'ScreenPorch',
#          'TotRmsAbvGrd',
#          'TotalBsmtSF',
#          'WoodDeckSF',
#          'YearBuilt',
#          'YearRemodAdd'
#     ],
#     nominal = [
#         'BldgType',
#         'BsmtCond',
#         'BsmtExposure',
#         'BsmtFinType1',
#         'BsmtFinType2',
#         'BsmtQual',
#         'CentralAir',
#         'Condition1',
#         'Condition2',
#         'Electrical',
#         'ExterCond',
#         'ExterQual',
#         'Exterior1st',
#         'Exterior2nd',
#         'FireplaceQu',
#         'Foundation',
#         'Functional',
#         'GarageCond',
#         'GarageFinish',
#         'GarageQual',
#         'GarageType',
#         'Heating',
#         'HeatingQC',
#         'HouseStyle',
#         'KitchenQual',
#         'LandContour',
#         'LandSlope',
#         'LotConfig',
#         'LotShape',
#         'MSZoning',
#         'Neighborhood',
#         'PavedDrive',
#         'RoofMatl',
#         'RoofStyle',
#         'SaleCondition',
#         'SaleType',
#         'Street',
#         'Utilities',
#         'MSSubClass'
#     ],
#     ordinal=['OverallQual', 'OverallCond', 'YrSold', 'MoSold']
# )
# feature_groups['discrete'] = feature_groups['nominal'] + feature_groups['ordinal']

In [ ]:
full_data = pd.concat([X_raw, data_test]) 
features = group_features(full_data)
full_clean_data = clean(full_data, features)
X = full_clean_data[lambda x: x.index.isin(X_raw.index)]
X_test = full_clean_data[lambda x: x.index.isin(data_test.index)]

In [ ]:
X.shape, X_test.shape

In [ ]:
selected_features = make_mi_scores(X, y, 'auto').index[-10:].tolist()
selected_features

# Baseline Model
 Given the large number of categorical variables, let's start with random forest so we don't have to do any further encoding.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer, mean_absolute_error
from sklearn.model_selection import train_test_split, cross_validate, GroupKFold

In [ ]:
model = RandomForestRegressor(random_state=0)

In [ ]:
scores = cross_validate(
    model, 
    X, 
    y, 
    cv=10, 
    scoring='neg_mean_absolute_error',
    return_train_score=True,

)

In [ ]:
-scores['train_score'].mean(), scores['train_score'].std()

In [ ]:
-scores['test_score'].mean(), scores['test_score'].std()

# Submit

In [ ]:
model.fit(X, y)

In [ ]:
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': model.predict(X_test)})
output.to_csv('submission.csv', index=False)